In [ ]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", 
                           filename="./data/ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", 
                           filename="./data/ratings_test.txt")

In [ ]:
train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')

In [ ]:
# 데이터 전처리
    # 1. 비어 있는 리뷰 처리
    # 2. 한글외의 문자는 Nan 처리
    # 3. 같은 리뷰 제거  
    
#  학습 데이터와 테스트 데이터로 정형화
    # 4. x 와 y로 분리
    # 5. 단어로 토큰 
    # 6. 패딩
    
# 7. 모델 구조 설계
# 8. 모델 실행
# 9. 모델 검증

In [ ]:
# 데이터 전처리
    # 1. 비어 있는 리뷰 처리
    # 2. 한글외의 문자는 Nan 처리
    # 3. 같은 리뷰 제거

# 1. 비어 있는 리뷰 처리   
train_data = train_data.dropna(how='any')  # null 존재하는 행 제거 
# 2. 한글을 제외한 문자를 '' 로 변경
train_data['document'] = train_data['document'].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 " "]', 
                                                          '', regex=True)
# '' 를 np.nan  으로 변경 -> nan 을 제거
train_data['document'].replace('', np.nan, inplace=True)
train_data.dropna(inplace=True)
# 3. 같은 리뷰 제거
train_data.drop_duplicates(subset=['document'], inplace=True) 

In [ ]:
#  학습 데이터와 테스트 데이터로 정형화
    # 4. x 와 y로 분리
    # 5. 단어로 토큰 
    # 6. 패딩

# 4. x 와 y로 분리
x_train = train_data['document']
y_train = train_data['label']

x_test = test_data['document']
y_test = test_data['label']



In [ ]:
## 딥러닝을 활용한 자연어 처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical


In [ ]:
# 5. 자연어 전처리 -> 토큰화
token = Tokenizer()
token.fit_on_texts(x_train) 

# 6. 서로 다른 길이의 데이터를 padding -> 10로 
padd_x = token.texts_to_sequences(x_train)   # 단어를 토큰화 하고 인덱스만 가져옴

# 서로 다른 길이의 데이터를 padding -> 10로 
padded_x = pad_sequences(padd_x, 10)

In [ ]:
# 임베딩에 입력된 단어의 수를 지정
word_size = len(token.word_index) + 1

# 7. 단어 임베딩을 포함한 딥러닝 모델 생성
model = Sequential()
model.add(Embedding(word_size, 8, input_length=10))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

#  8. model 실행 옵션 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_x, y_train, epochs=20)

In [ ]:
# 9. 검증하기 위한 테스트 자료
# 서로 다른 길이의 데이터를 padding -> 10로 
padd_xt = token.texts_to_sequences(x_test)   # 단어를 토큰화 하고 인덱스만 가져옴

# 서로 다른 길이의 데이터를 padding -> 10로 
padded_xt = pad_sequences(padd_xt, 10)
model.evaluate(padded_xt, y_test)